## Populate Task

In [1]:
import sys
from pathlib import Path

# Đường dẫn tới thư mục gốc của dự án
project_root = Path('C:/Users/ADMIN/Desktop/DATN/Extract_Information')
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

In [2]:
task = 'explicit'
model_name = 'GPT'

In [3]:
PATH_FILE_TASK = 'C:/Users/ADMIN/Desktop/DATN/Extract_Information/tasks/task_template/'
PATH_FILE_TEMPLATE = 'C:/Users/ADMIN/Desktop/DATN/Extract_Information/tasks/tasks/task_template.json'
PATH_FILE_DATA = 'C:/Users/ADMIN/Desktop/DATN/Extract_Information/data/mave_filtered_test.jsonl'

if model_name == 'GPT':
    PATH_FILE_RESULT = 'C:/Users/ADMIN/Desktop/DATN/Extract_Information/tasks/result/GPT'
else:
    PATH_FILE_RESULT = 'C:/Users/ADMIN/Desktop/DATN/Extract_Information/tasks/result/Llama'

In [4]:
from dotenv import load_dotenv

# Load OPENAI_API_KEY from .env file
load_dotenv()

True

In [5]:
import json
from pieutils.pieutils import update_task_dict_from_file, combine_example, calculate_recall_precision_f1

# Load task template
with open(PATH_FILE_TEMPLATE, 'r') as f:
    task_dict = json.load(f)

task_dict['task_prefix'] = "Extract the exact attribute value of the attribute from the following " \
                            "product title and return only the attribute value. If the attribute " \
                            "value is not contained, respond with 'I do not know.'."

task_dict['shots'] = 0

# Generate examples
task_dict = update_task_dict_from_file(PATH_FILE_DATA, task_dict, True)

with open(PATH_FILE_TASK + 'task_{}.json'.format(task), 'w', encoding='utf-8') as f:
    json.dump(task_dict, f, indent=4)

## Evaluate Task

In [72]:
# Initialize processing
from tqdm import tqdm
import torch
import transformers
from langchain import LLMChain, HuggingFacePipeline
from langchain.chat_models import ChatOpenAI

from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.callbacks import get_openai_callback

In [ ]:
# Model
if model_name == 'GPT':
    llm = ChatOpenAI(model_name='gpt-3.5-turbo-0125', temperature=0)
else:
    #Llama 3
    tokenizer = AutoTokenizer.from_pretrained("/ceph/alebrink/cache/models--meta-llama--Meta-Llama-3-8B-Instruct/snapshots/c4a54320a52ed5f88b7a2f84496903ea4ff07b45")
    model = AutoModelForCausalLM.from_pretrained(
        '/ceph/alebrink/cache/models--meta-llama--Meta-Llama-3-8B-Instruct/snapshots/c4a54320a52ed5f88b7a2f84496903ea4ff07b45',
        torch_dtype=torch.float16,
        device_map="auto",
        offload_folder="save_folder",
    )
    model.tie_weights()

    hf_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        device_map="auto"
    )

    terminators = [
        hf_pipeline.tokenizer.eos_token_id,
        hf_pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    llm = HuggingFacePipeline(pipeline=hf_pipeline)

In [ ]:
# Put task prefix into system message.
prompt_list = []
system_message_prompt = SystemMessagePromptTemplate.from_template(task_dict['task_prefix'])
prompt_list.append(system_message_prompt)

# Add 
human_template="{input_string}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
prompt_list.append(human_message_prompt)

chat_prompt = ChatPromptTemplate.from_messages(prompt_list)

llm_chain = LLMChain(
    prompt=chat_prompt,
    llm=llm
)

# Save model
task_dict['model'] = model_name

In [ ]:
def select_and_run_llm(attribute, input):
        pred = None
        if len(input) == 0:
            # No input text provided.
            return pred
        
        if model_name == 'GPT':
            response = llm_chain.run(input_string = input)
        else:    
            messages = [{"role": "system", "content": chat_prompt.messages[0].format().content},
                        {"role": "human", "content": chat_prompt.messages[1].format(input_string = input).content}]

            hf_prompt = hf_pipeline.tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )

            hf_outputs = hf_pipeline(hf_prompt, max_new_tokens=256,
                                    eos_token_id=terminators,
                                    do_sample=True,
                                    temperature=0,
                                    top_p=0.9
                                    )
            response = hf_outputs[0]["generated_text"][len(hf_prompt):]
        
        try:
            print(response)
            pred ={}
            pred[attribute] = response
        except Exception as e:
            print(e)
            print('Response: ')
            print(response)
        return pred

In [ ]:
if model_name == 'GPT':
    with get_openai_callback() as cb:
        preds = [select_and_run_llm(example['attribute'], example['input']) for example in tqdm(task_dict['examples'])]
    
    task_dict['total_tokens'] = cb.total_tokens
    print(f"Total Tokens: {cb.total_tokens}")
else:
    preds = [select_and_run_llm(example['attribute'], example['input']) for example in tqdm(task_dict['examples'])]

In [ ]:
print(preds)

[{'SD Format': 'Micro SDHC'}, {'SD Format': 'MicroSDHC'}, {'SD Format': 'custom'}, {'SD Format': 'Standard'}, {'Capacity': '4GB'}, {'Capacity': '4GB'}, {'Capacity': '32GB'}, {'SD Format': 'Standard'}, {'Capacity': '64GB'}, {'SD Format': 'Standard'}, {'SD Format': 'MicroSDHC'}, {'SD Format': 'MicroSDHC'}, {'SD Format': 'MicroSDHC'}, {'Capacity': '16 MB'}, {'Optical Zoom': '3x'}, {'Capacity': '64 MB'}, {'Capacity': '64 MB'}, {'Optical Zoom': '3x'}, {'Resolution': '2.3 MP'}, {'Optical Zoom': '3x'}, {'Capacity': '128 MB'}, {'Optical Zoom': '3x'}, {'Resolution': '3.3 MP'}, {'Optical Zoom': '4x'}, {'Optical Zoom': '10x'}, {'Capacity': '256 MB'}, {'Capacity': '128 MB'}, {'Resolution': '3.1 MP'}, {'Optical Zoom': '3x'}, {'Optical Zoom': '12x'}, {'Resolution': '4MP'}, {'Optical Zoom': '3x'}, {'Resolution': '3.2 MP'}, {'Optical Zoom': '6x'}, {'Optical Zoom': '3x'}, {'Optical Zoom': '5x'}, {'Optical Zoom': '3x'}, {'Resolution': '7.1MP'}, {'Optical Zoom': '12x'}, {'Resolution': '1024 x 768'}, {'Ca

In [ ]:
targets = [example['target_scores'] for example in task_dict['examples']]
categories = [example['category'] for example in task_dict['examples']]
attributes = [example['attribute'] for example in task_dict['examples']]

postprocessed_preds = [pred[attribute] for pred, attribute in zip(preds, attributes)]
#postprocessed_preds = [example['post_pred'] for example in task_dict['examples']]

task_dict['examples'] = [combine_example(example, pred, post_pred) 
                    for example, pred, post_pred in zip(task_dict['examples'], preds, postprocessed_preds)]

results = calculate_recall_precision_f1(targets, postprocessed_preds, categories, attributes)

Attribute: SD Format - Category: Flash Memory Cards
{'precision': 74.29, 'recall': 63.41, 'f1': 68.42}
Attribute: Capacity - Category: Flash Memory Cards
{'precision': 100.0, 'recall': 100.0, 'f1': 100.0}
Attribute: Optical Zoom - Category: Digital Cameras
{'precision': 100.0, 'recall': 100.0, 'f1': 100.0}
Attribute: Sensor Size - Category: Digital Cameras
{'precision': 100.0, 'recall': 50.0, 'f1': 66.67}
Attribute: Resolution - Category: Digital Cameras
{'precision': 100.0, 'recall': 97.56, 'f1': 98.76}
Attribute: Sensor Type - Category: Digital Cameras
{'precision': 97.44, 'recall': 95.0, 'f1': 96.2}
Attribute: Battery Life - Category: Laptops
{'precision': 85.71, 'recall': 85.71, 'f1': 85.71}
Attribute: Processor Speed - Category: Laptops
{'precision': 95.24, 'recall': 90.91, 'f1': 93.02}
Attribute: Processor Brand - Category: Laptops
{'precision': 91.3, 'recall': 95.45, 'f1': 93.33}
Attribute: Screen Size - Category: Laptops
{'precision': 97.56, 'recall': 97.56, 'f1': 97.56}
Attrib

In [ ]:
# Save populated task
result_file = PATH_FILE_RESULT + '{}_{}.json'.format(task, task_dict['model'])
with open(result_file, 'w') as fp:
    json.dump(task_dict, fp, indent=4)

In [ ]:
# Error Analysis
print('Prompts for which target and postprocessed prediction do not match.')
print('-----------')
input_texts = [example['input'] for example in task_dict['examples']]

for input_text, target, pred, post_pred in zip(input_texts, targets, preds, postprocessed_preds):
    if post_pred not in target.keys():
        #print('Prompt: {}'.format(chat_prompt.format(task_prefix=task_dict['task_prefix'], input_string= input_text, human_msg_0= human_text_0, ai_msg=system_text,)))
        print('Input: {}'.format(input_text))
        print('Prediction: \n {}'.format(pred))
        print('Prediction 2: {}'.format(post_pred))
        print('Target: {}'.format(target))
        print('-----------')

Prompts for which target and postprocessed prediction do not match.
-----------
Input: Essential ULTRA 32GB Best Buy Insignia Flex MicroSDHC Card with custom format for Hi-Speed Lossless certified recording! With SD Adapter. (Class 10, up to 500x or 70MB/sec) 
 Requested Attribute: SD Format
Prediction: 
 {'SD Format': 'custom'}
Prediction 2: custom
Target: {'MicroSDHC Card': 1, 'MicroSDHC': 1}
-----------
Input: Professional Ultra SanDisk 32GB MicroSDHC Card for Nokia Lumia 521 Smartphone is custom formatted for high speed, lossless recording! Includes Standard SD Adapter. (UHS-1 Class 10 Certified 30MB/sec) 
 Requested Attribute: SD Format
Prediction: 
 {'SD Format': 'Standard'}
Prediction 2: Standard
Target: {'MicroSDHC Card': 1, 'MicroSDHC': 1}
-----------
Input: Professional Ultra SanDisk 64GB MicroSDXC LG G Pad 8.3 card is custom formatted for high speed, lossless recording! Includes Standard SD Adapter. (UHS-1 Class 10 Certified 30MB/sec) 
 Requested Attribute: SD Format
Predict